In [ ]:
import glob
import pandas as pd
import numpy as np
import seaborn as sns
import sys
import matplotlib.pyplot as plt
# sys.path

importación glob
importar pandas como pd
importar numpy como np

# Llenado de valores faltantes

**Objetivo**: El objetivo de esta sección es mostrar algunas de las técnicas que pueden ser usadas para rellenar datos faltantes. En este caso de estudio abordaremos los métodos `fillna` e `interpolate` de la librería `pandas`.

## Calidad del aire de Beijing

El caso de estudio se centra en un conjunto de datos de la siguiente publicación

- \[1\] [Zhang, S., Guo, B., Dong, A., He, J., Xu, Z. and Chen, S.X. (2017) Cautionary Tales on Air-Quality Improvement in Beijing. Proceedings of the Royal Society A, Volume 473, No. 2205, Pages 20170457.](https://royalsocietypublishing.org/doi/pdf/10.1098/rspa.2017.0457)

Disponible en el repositorio de [UCI](https://archive.ics.uci.edu/ml/datasets/Beijing+Multi-Site+Air-Quality+Data)

Las columnas incluidas en este conjunto de datos son las siguientes:

- No: número de fila
- year: año de datos en esta fila
- month: mes de datos en esta fila
- day: día de datos en esta fila
- hour: hora de datos en esta fila
- PM2.5: concentración de PM2.5 (ug / m ^ 3)
- PM10: concentración de PM10 (ug / m ^ 3)
- SO2: concentración de SO2 (ug / m ^ 3)
- NO2: concentración de NO2 (ug / m ^ 3)
- CO: concentración de CO (ug / m ^ 3)
- O3: concentración de O3 (ug / m ^ 3)
- TEMP: temperatura (grado Celsius)
- PRES: presión (hPa)
- DEWP: temperatura de rocío (grados Celsius) - temperatura de rocío es la más alta temperatura a la que empieza a condensarse el vapor de agua contenido en el aire.
- RAIN: precipitación (mm)
- wd: dirección del viento
- WSPM: velocidad del viento (m / s)
- station: nombre del sitio de monitoreo de la calidad del aire

Las estaciones están todas ubicadas en el área metropolitana de Beijing. En la siguiente figura los puntos violetas muestran su ubicación, los triángulos son estaciones meteorológicas y los puntos rojos son otras estaciones no incluidas en el conjunto de datos. Las lineas azul y naranjas fividen a la metropolis en zona norte, centro y sur.     

<img src="img/Beijing_AQ.png" style='width:600px'>
<center><b>Figura 1:</b> Ubicación de las estaciones de monitoreo - Fuente [1].</center>

**Consolidar conjunto de datos**  
En primer lugar cargamos el conjunto de datos en memoria. Cada csv contiene las series de tiempo con frecuencia horaria de las 12 estaciones mencionadas, para 4 años calendario, desde marzo 2013 hasta febrero 2017. Por este motivo, todos los archivos tienen la misma cantidad de filas y columnas.

Para trabajar todo el conjunto de datos en un solo archivo, construimos un `DataFrame` con un multiíndice en las columnas. En el primer nivel usaremos el nombre de la estación y en la segunda las variables anteriormente descritas. 

In [ ]:
# contruye una lista con las rutas de los archivos csv a cargar
rutas_csv = sorted(glob.glob('data_UCI/PRSA_Data_20130301-20170228/*'))

# inicializa el diccionario que contendra los DataFrame de cadad estacion
df_dict = {}

for csv_p in rutas_csv:
    
    # reporta el archivo cargado y su numero de filas y columas
    print(f'cargando el archivo: {csv_p}')
    df_ = pd.read_csv(csv_p, index_col=0)
    print(f'shape: {df_.shape}\n')
    
    # obten el nombre de la estacion ara usarla como llave del diccionario
    estacion_arreglo = df_.station.unique()
    estacion_name = estacion_arreglo[0]

    df_dict[estacion_name] = df_

# construye el DataFrame consolidado 
baq_df = pd.concat(df_dict, axis=1)
baq_df

**Contar número de valores nulos**  
Para mostrar las diferentes maneras de llenado de valores faltantes primero estudiaremos la cantidad de valores nulos que presentan los datos.

In [ ]:
# cuenta el numero de valores nulos y agrupa por columna en el segundo nivel
grupo = baq_df.isnull().sum().groupby(level=1, sort=False)

# suma los valos nulos de cada grupo
n_nulos = grupo.sum()

# reporta los resultados en numero absoluto y porcentaje 
df_nulos = pd.concat([n_nulos, n_nulos * 100 / (len(df_dict) * baq_df.shape[0])],
                     keys=['n_nulos', 'porcentaje'], axis=1)
df_nulos

**Contar la cantidad máxima de valores nulos contiguos**  
Además de contar la cantidad de nulos, dado que se trata de series de tiempo, nos interesa saber la cantidad máxima de valores nulos contiguos.

In [ ]:
def encontrar_max_contiguo(srs):
    '''
    Encuentra la maxima suma de `1` adyacentes en una
    en un `iterable` de `0` y `1`
    
    Parametros
    ----------
    srs : iterable,
        compuesto solo por elementos `0` o `1`.
        
    Retorna
    -------
    max : int,
        el numero maximo de `1` contiguos
    '''
    
    # inicializa valores maximos
    max_ = 0
    contiguo_max = 0
    
    # recorre los
    for a in srs:
        if a != 0:
            contiguo_max += a
            if max_ < contiguo_max:
                max_ = contiguo_max
        else:
            contiguo_max = 0
    return max_

In [ ]:
# calcula cantidad maxima de valores nulos contiguos en las
# columnas que presentan valores nulos
nulos_contiguos = baq_df.isnull().loc[:, baq_df.isnull().any(0)].apply(encontrar_max_contiguo)
nulos_contiguos

In [ ]:
# calcula  el promedio de nulos contiguos por variable
grupo_nulos_contiguos = nulos_contiguos.groupby(level=1, sort=False)
grupo_nulos_contiguos.describe()

### Método `fillna`
Para las columnas en las que el porcentaje de nulos es menor al 1% llenaremos los valores faltantes con métodos simples que están disponibles en el método `fillna`. Además, en estas columnas los valores faltantes consisten en a lo más un periodo de 13 horas y en su mayoría no sobrepasa las 6 horas corridas (ver celda anterior).

In [ ]:
# este objeto sera util para trabajar con índices multiples
idx = pd.IndexSlice

# muestra en pantalla las variables que presentan valores nulos menores al 1%
df_nulos.index[(df_nulos['porcentaje'] < 1) & (df_nulos['porcentaje'] > 0)]

**Llenado con una constante**  
Para la variable `'RAIN'` asumiremos arbitrariamente que si el valor es nulo, se debe a ausencia de lluvia

In [ ]:
# muestra en pantalla las filas con valores nulos
col_a_llenar = 'RAIN'
filas_con_nulos = baq_df.loc(axis=1)[:, col_a_llenar].isnull().any(1)
baq_df.loc[filas_con_nulos, idx[:, col_a_llenar]]

In [ ]:
# rellena los valores nulos
baq_df.loc(axis=1)[:, col_a_llenar] = baq_df.loc(axis=1)[:, col_a_llenar].fillna(0)
baq_df.loc[filas_con_nulos, idx[:, col_a_llenar]]

**Llenado con la moda**  
Para nuestra única variable categórica, `'wd'` (dirección del viento). 

In [ ]:
# muestra en pantalla las filas con valores nulos
col_a_llenar = 'wd'
filas_con_nulos = baq_df.loc(axis=1)[:, col_a_llenar].isnull().any(1)
baq_df.loc[filas_con_nulos, idx[:, col_a_llenar]]

In [ ]:
# calcula la moda por columna
moda_wd = baq_df.loc(axis=1)[:, col_a_llenar].mode()
moda_wd.iloc[0]

In [ ]:
# rellena los valores nulos
baq_df.loc(axis=1)[:, col_a_llenar] = baq_df.loc(axis=1)[:, col_a_llenar].fillna(
    moda_wd.iloc[0], axis=0)
baq_df.loc[filas_con_nulos, idx[:, col_a_llenar]]

**Llenado con la mediana**  
Para la columna `'WSPM'` (velocidad del viento). 

In [ ]:
# muestra en pantalla las filas con valores nulos
col_a_llenar = 'WSPM'
filas_con_nulos = baq_df.loc(axis=1)[:, col_a_llenar].isnull().any(1)
baq_df.loc[filas_con_nulos, idx[:, col_a_llenar]]

In [ ]:
# calcula la mediana por columna
mediana_wspm = baq_df.loc(axis=1)[:, col_a_llenar].median()
mediana_wspm

In [ ]:
# rellena los valores nulos
baq_df.loc(axis=1)[:, col_a_llenar] = baq_df.loc(axis=1)[:, col_a_llenar].fillna(
    mediana_wspm, axis=0)
baq_df.loc[filas_con_nulos, idx[:, col_a_llenar]]

**Llenado con el promedio**  
Para las columna `'PRES'` (presión).

In [ ]:
# muestra en pantalla las filas con valores nulos
col_a_llenar = 'PRES'
filas_con_nulos = baq_df.loc(axis=1)[:, col_a_llenar].isnull().any(1)
baq_df.loc[filas_con_nulos, idx[:, col_a_llenar]]

In [ ]:
# calcula la moda por columna
promedio_pres = baq_df.loc(axis=1)[:, col_a_llenar].mean()
promedio_pres

In [ ]:
# rellena los valores nulos
baq_df.loc(axis=1)[:, col_a_llenar] = baq_df.loc(axis=1)[:, col_a_llenar].fillna(
    promedio_pres, axis=0)
baq_df.loc[filas_con_nulos, idx[:, col_a_llenar]]

**Llenado con métodos `'ffill'` y `'bfill'`**  

El método `fillna` tiene dos métodos que son útiles para DataFrame's donde que dos valores sean contiguos implica alguna noción de proximidad. En el caso de estudio, al tratarse de series de tiempo, dos valores contiguos  ocurren cerca el espacio temporal. Estos métodos son:
1. Llenado hacia adelante (`'ffill'`): donde el último valor no nulo es propagado hacia adelante
2. Llenado hacia atrás (`'bfill'`): donde los valores nulos son reemplazados por el siguiente valor no nulo.
  
<bk>  
    
- Así, para la columna `'Temp'` (temperatura), usaremos el método de llenado hacia adelante

In [ ]:
# muestra en pantalla las filas con valores nulos
col_a_llenar = 'TEMP'
filas_con_nulos = baq_df.loc(axis=1)[:, col_a_llenar].isnull().any(1)
baq_df.loc[filas_con_nulos, idx[:, col_a_llenar]]

In [ ]:
# rellena los valores nulos
baq_df.loc(axis=1)[:, col_a_llenar] = baq_df.loc(axis=1)[:, col_a_llenar].fillna(
    method='ffill')
baq_df.loc[filas_con_nulos, idx[:, col_a_llenar]]

- Por su parte para la columna `'DEWP'`, usaremos el método de llenado hacia atrás.

In [ ]:
# muestra en pantalla las filas con valores nulos
col_a_llenar = 'DEWP'
filas_con_nulos = baq_df.loc(axis=1)[:, col_a_llenar].isnull().any(1)
baq_df.loc[filas_con_nulos, idx[:, col_a_llenar]]

In [ ]:
# rellena los valores nulos
baq_df.loc(axis=1)[:, col_a_llenar] = baq_df.loc(axis=1)[:, col_a_llenar].fillna(
    method='bfill')
baq_df.loc[filas_con_nulos, idx[:, col_a_llenar]]

**Comprobar el procedimiento**  
Ahora que ya hemos llenado los valores nulos de todas las columnas que presentaban un porcentaje menor a 1%, comprobemos que esto se ha ejecutado satisfactoriamente, calculando nuevamente los valores faltantes por columna.

In [ ]:
# cuenta el numero de valores nulos y agrupa por columna en el segundo nivel
grupo_despues_fillna = baq_df.isnull().sum().groupby(level=1, sort=False)

# suma los valos nulos de cada grupo
n_nulos_despues_fillna = grupo_despues_fillna.sum()

# reporta los resultados en numero absoluto y porcentaje 
df_nulos_despues_fillna = pd.concat([n_nulos_despues_fillna, n_nulos_despues_fillna * 100 / (len(df_dict) * baq_df.shape[0])],
                     keys=['n_nulos', 'porcentaje'], axis=1)
print('antes fillna'); display(df_nulos)
print('\n\ndespues fillna'); display(df_nulos_despues_fillna)

### Método `interpolate`
Los valores que faltantes restantes, serán rellenados mediante el método `interpolate` (su nombre es autoexplicativo).
- Se podría estudiar cual es la interpolación que tiene mejor fit con cada una de los contaminantes. (**Ejercicio?:** 80, 20 para train y test y medida de performance RMSE.)


In [ ]:
test_df = baq_df.loc[:, baq_df.isnull().any(0)].copy()

In [ ]:
test_df.interpolate(method='linear')

# BREAK
# NO INCLUIR (Puede servir de base para MOGP)
El objetivo de la publicación \[1\] [Zhang, S., Guo, B., Dong, A., He, J., Xu, Z. and Chen, S.X. (2017) Cautionary Tales on Air-Quality Improvement in Beijing. Proceedings of the Royal Society A, Volume 473, No. 2205, Pages 20170457.](https://royalsocietypublishing.org/doi/pdf/10.1098/rspa.2017.0457) es calcular el promedio de PM2.5 en el año 2017.  
- se puede estudiar el modelo de regresión propuesto en el paper
- se puede usar MOGP para calcular dicho promedio, tomando la esperanza en el área de la ciudad completa y con un intervalo de confianza. 

In [ ]:
estacion_max_nul = baq_df.isnull().groupby(axis=1, level=0).sum().sum().idxmax()

In [ ]:
# df = baq_df.xs(estacion_max_nul, axis=1, level=0).copy()
df = df_dict[estacion_max_nul]
display(df)

**Verificar que se tienen 4 años de datos**

In [ ]:
columnas_temp = ['year', 'month', 'day', 'hour']
for ct in columnas_temp:
    print(f'\nvalue counts para columna {ct}')
    display(df[ct].value_counts(sort=False).sort_index())

**Visualizar la distribución de los valores nulos**

In [ ]:
nulos_df = df.loc[:, df.isnull().any(0)]

In [ ]:
plt.subplots(figsize=(8, 8))
sns.heatmap(~ nulos_df.isnull(), cbar=False)
plt.title('Matriz de presencia de nulos');

In [ ]:
# Calcular matriz de correlación de presencia de nulos
nulos_corr = nulos_df.isnull().corr()

def dibuja_matriz_de_correlación(corr_df):
    '''Funcion personalizada para dibugjar la priz de correlación'''
    sns.set(style="white", font_scale=1.25)

    # Generar una mascara para el triangulo superior
    mascara = np.triu(np.ones_like(corr_df.values, dtype=np.bool))

    # Iniciaizar fugura para configurar tamanño
    f, ax = plt.subplots(figsize=(10, 8))

    # Generar un colormap apropiado
    cmap = sns.diverging_palette(220, 10, as_cmap=True)

    # Dibujar matriz de correlación
    sns.heatmap(corr_df.values, mask=mascara, cbar=True, cmap=cmap, vmin=-1, vmax=1,
                annot=True, square=True, fmt='.2f', linewidths=.5, 
                cbar_kws={"shrink": .5}, yticklabels=corr_df.columns.values, 
                xticklabels=corr_df.columns.values)
    

dibuja_matriz_de_correlación(nulos_corr)
plt.title('Correlación de la matriz de presencia de nulos')

La matriz nos indica la cuan fuertemente la presencia o ausencia de una variable afecta la presencia de la otra.  
- si dicha correlación es cercana a 1, la presencia de una variable *asegura* la presencia de la otra.
- si es cercana a -1, la presencia de una variable *asegura* la ausencia de la otra
- si es cercana a 0, la presencia de una variable no tiene relación con la presencia de la otra.

Así da la impresión que los censores fuertemente relacionados en cuanto a presencia de valores faltantes son:
1. Sensores de tempreatura, presión, temperatura de rocío y velocidad del viento.
2. Los sensores de PM10 y PM2.5
3. Los sensores de CO y NO2

En cuanto a correlaciones de presencia de valores faltantes de mediana intensidad:

1. Los sensores de contaminantes entre ellos con excepción de los pares mencionados arriba.
2. El sensor de dirección de viento con el sensor de velocidad del viento.

Además podemos inferir gracias al rectángulo gris que la presencia de valores faltantes en los censores de los contaminantes (PM2, PM10, SO2, NO2, CO y O3) no tiene relación con la presencia de valores faltantes en los de condiciones atmosféricas (el resto de variables). 

Nos podríamos preguntar si esa última observación también se observa en la matriz de correlación de los valores observados en dichas variables

In [ ]:
# Calcular matriz de correlacion
corr = nulos_df.corr()

# dibuja matriz de correlación
dibuja_matriz_de_correlación(corr)
plt.title('Matriz de correlacion')

Ahora podemos observar que existe una correlación importante entre la mayoría de los contaminantes, excepto el ozono (O3), que por suparte presenta una correlación importante con la temperatura.
- Propongo hacer un MOGP con los contaminantes y la temperatura

# NO INCLUIR
**Estudio de nulos valores de temperatura**

In [ ]:
temp_df = pd.concat(df_dict, axis=1).xs('TEMP', axis=1, level=1)
plt.subplots(figsize=(8, 8))
sns.heatmap(temp_df.isnull(), cbar=False, linewidth=0)
plt.title('Matriz de presencia de nulos: columnas `TEMP`');

In [ ]:
plot_srs = temp_df.isnull().sum()
plot_srs.plot(kind='bar', color='grey')
for i, patch in enumerate(plt.gca().patches):
    
    # obtener coordenadas y medidas
    x = patch.get_x()
    h = patch.get_height()
    w = patch.get_width()
    
    # adjuntar texto con valores de cada barra
    plt.annotate(f'{plot_srs.iloc[i]}', xy=(x + w / 2, h), ha='center', va='bottom')
    
plt.ylim([1.1 * y_ for y_ in plt.gca().get_ylim()])

In [ ]:
dibuja_matriz_de_correlación(temp_df.isnull().corr())
plt.title('Correlación de la matriz de presencia de nulos en TEMPERATURA')

# Expresiones regulares
Limpieza de texto con expresiones regulares.

## Datos de Twitter
La [Competencia Kaggle: Twitter Sentiment Extraction](https://www.kaggle.com/c/tweet-sentiment-extraction) está actualmente en linea (el premio es 15 mil dólares!) y se trata de encontrar es substring del tweet que contiene el sentimiento del tweet completo.

El objetivo de este pequeño ejercicio es simplemente dar una introducción a las **expresiones regulares**, ya que son muy útiles como herramienta para la limpieza de texto.

**Importar librerias**  
La librería para el uso de expresiones regulares en Python es `re`.

In [ ]:
# import re
import string 

**Cargar datos**  
El conjunto de datos de entrenamiento tiene 4 columnas:
- textID: indetificar del tweet
- text: contenido del tweet
- selected_text: extracto del tweet que expresa el sentimiento
- sentiment: sentimiento del tweet. Puede ser `'negative'`,`'neutral'`,`'positive'`

Naturalmente, las columnas que nos interesan limpiar son `'text'`y `'selected_text`. 

In [ ]:
ruta_tse_entrenamiento = 'data_kaggle/twitter/train.csv'
tse_df = pd.read_csv(ruta_tse_entrenamiento)

In [ ]:
tse_df

In [ ]:
srs = tse_df['text']

- convertir el texto en minúsculas

In [ ]:
srs = srs.str.lower()
srs

In [ ]:
def eliminar_con_regex(srs, regex, verbose=True):
    '''Funcion personalizada para eliminar texto de la serie `srs` 
    mediante la expresión regular `regex`.
    '''
    
    if verbose:
        
        # encunetra filas que coinciden con la expresión regular
        bools_ = srs.str.contains(regex, na=False)
        
        # reporta las filas que tienen coincidencia 
        print('antes de eliminar:')
        display(srs[bools_])
    
    # elimina el substring
    ret = srs.str.replace(regex, '')
    
    if verbose:
        
        # reporta las filas que tienen coincidencia 
        print('\n\ndespues de eliminar:')
        display(ret[bools_])
        
    return ret

- quitar texto entre paréntesis cuadrados, pues estos son usados para denotar un lugar geolocalizado. Para esto usamos la expresión regular `'\[.*?\]'`. Esta está compuesta por:
    - `'\['` y `\]` que denotan simplemente los caracteres `'['` y `']'`. Esto debido a que para las expresiones regulares, todos los carácteres hacen coincidencia con si mismos salvo `'+'`, `'?'`, `'.'`, `'*'`, `'^'`, `'$'`, `'('`, `')'`, `'['`, `']'`, `'{'`, `'}'`, `'|'` y `'\'` que son caracteres especiales.
    - `'.'` que denota cualquier carácter, excepto la nueva línea.
    - `'*'` es un cuantificador de 0 o más ocurrencias de la expresión precedente. Cunado un cuantificador es sucedido por un `'?'` se transforma en un cuantificado flojo, lo que implica que a penas la expresión regular completa haga coincidencia, no seguirá buscando otra que la contenga. Por ejemplo en el string `'10100000100'` la expresión regular `'1.*?1'` entregara el substring `'101'` como coincidencia. Por su parte, con la expresión regular `'1.*1'` entregará `'101000001'`  
    
  Por lo tanto, la expresión denota cualquier texto entre paréntesis cuadrados que no contenga un salto de línea.
  

In [ ]:
srs = eliminar_con_regex(srs, '\[.*?\]');

- Eliminar los links: Para ello se usa la expresión regular `'https?://\S+|www\.\S+'` que está compuesta por:
    - `'?'` es un cuantificador que acepta 0 o 1 ocurrencia de la expresión regular que la antecede. En este caso 0 o 1 ocurrencias de `'s'`.
    - `'\S'` coincide con todo caracter que no sea espacio (i.e. `'\t'`, `'\r'`, `'\n'` o `'\f'`)
    - `'+'` es un cuantificador que acepta 1 o más ocurrencias de la expresión regular que la antecede (patricularmente `'\S'`)
    - `'|'` denota un **o**, es decir `'https?://\S+'` **o** `'www\.\S+'`
    - `'\.'` denota `'.'`, pues este es un carácter especial. Podemos inferir que `'\'` se usa como carácter de escape.

In [ ]:
srs = eliminar_con_regex(srs, 'https?://\S+|www\.\S+');

- Eliminar todos los caracteres de puntuación. Para ellos usamos el atributo `punctuation` de la libreria `string`. Además se usan los carácteres especiales `'['` y `']'` que permiten especificar todos los posibles valores que puede tomar un carácter para coincidir con la expresión regular, así `'[python]'` es una expresión regular que coincide con los caracteres `'h'`,  `'n'`, `'o'`, `'p'`, `'t'` o `'y'`.  Notamos también que no es necesario usar el carácter de escape excepto para `'['` y `']'` para los demás caracteres especiales.

In [ ]:
from string import punctuation
punctuation

In [ ]:
srs = eliminar_con_regex(srs, f'[{punctuation}]');

- Eliminar las palabras que contienen dígitos. Para ello usamos la expresión `'\w*\d\w*'`. Esta está compuesta por:
    - `'\w'` coincide con todos los caracteres que pueden componer palabras (alfanumericos y guíon bajo).
    - `'\d'` coincide con dígitos (equivalente a `'[0-9]'`) 

In [ ]:
srs = eliminar_con_regex(srs, '\w*\d\w*');

In [ ]:
tse_df.loc[:, 'text'] = srs


# Outliers



## IQR
**Construyendo el puntaje IQR**

El puntaje IQR es un método común para eliminar valores atípicos (u *outliers*). El siguiente diagrama ilustra cómo se comporta el puntaje IQR en una población distribuida normalmente

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/1a/Boxplot_vs_PDF.svg/800px-Boxplot_vs_PDF.svg.png" alt="Stages 1" style="width: 450px;" align="center" frameborder="200"/>

<bk>  
    
    
<center><b>Fuente:</b> 
     <a href="https://en.wikipedia.org/wiki/Interquartile_range">Wikipedia Interquartile range webpage</a> 
</center>

Esta método de detección de valores atípicos es útil ya que depende de estadísticos robustos a la presencia de los mismos.  
    
Definimos la siguiente función que trabaja sobre valores de Series.

In [ ]:
# función personalizada para construir la puntuación IQR
def q1 (srs):
    """
    Calcula el primer cuartil de una serie.
    """
    return srs.quantile (0.25)

In [ ]:
def q3 (srs):
    """
    Calcula el primer cuartil de una serie.
    """
    return srs.quantile(0.75)

In [ ]:
def calcula_puntaje_RIC(df_, col, verbose=True):
    """Agrega una columna de puntaje de rango intercuartilico a `df`, 
    basado en una columna que que contiene `col` como columna.

    Parametros
    ----------
    df : pd.DataFrame
        donde se agregará la columna de puntaje RIC
    col : inmutable, 
        define la columna en la que se obtendrá la puntuación RIC 
        calculado
    verbose : bool, 
        ya sea que muestre o no el calculo del RIC

    Retorna
    ----------
    Copia de `df` con la columna de ``PRIC_{`col`}``
    """
    
    # crea una copia del data frame 
    df = df_.copy()

    # calcula cuartiles 1 y 3
    ric_df = df[[col]].aggregate([q1, q3])

    # calcula RIC
    ric_df.loc['RIC'] = ric_df.loc['q3'].iloc[0] - ric_df.loc['q1'].iloc[0]

    if verbose:
        # print('RIC:')
        display(ric_df)

    # inicializa la columna de puntaje en cero
    df[f'PRIC_{col}'] = 0

    # identifica filas menores a Q1
    bools_menor_Q1 = (df[col] <= ric_df.loc['q1'].iloc[0])
    sel_idx_Q1 = df.index[bools_menor_Q1]

    # identifica filas mayores a Q3 
    bools_mayor_Q3 = (df[col] >= ric_df.loc['q3'].iloc[0])
    sel_idx_Q3 = df.index[bools_mayor_Q3]
    
    # calcula puntaje RIC para filas menores a Q1
    df.loc[sel_idx_Q1, f'PRIC_{col}'] = (
        ric_df.loc['q1'].iloc[0] - df.loc[sel_idx_Q1, col].values) / \
        ric_df.loc['RIC'].iloc[0]

    # calcula puntaje RIC para filas menores a Q1
    df.loc[sel_idx_Q3, f'PRIC_{col}'] = (
        df.loc[sel_idx_Q3, col].values - ric_df.loc['q3'].iloc[0]) / \
        ric_df.loc['RIC'].iloc[0]

    return df

In [ ]:
!unzip "data_UCI/household_power_consumption.txt.zip"

In [ ]:
electricidad_df = pd.read_csv('household_power_consumption.txt', sep=';', na_values='?')

In [ ]:
for j in range(2, electricidad_df.shape[1]):
    plt.figure()
    sns.boxplot(electricidad_df.iloc[:, j])
    plt.show()

Dado el plot de arriba, lo usaría solamente en 2 a 6.

In [ ]:
df_ = None
for col in electricidad_df.columns[2:6]:
    if df_ is None:
        df_ = calcula_puntaje_RIC(electricidad_df, col)
        
    else:
        df_ = calcula_puntaje_RIC(df_, col)

In [ ]:
display(df_)

## STD
[El notebook de kaggle más votado](https://www.kaggle.com/pmarcelino/comprehensive-data-exploration-with-python): interés especial en la sección **Out Liars!** se usa un criterio de std  

El dataset ya está descargado en:

In [ ]:
pd.read_csv('data_kaggle/house-prices-advanced-regression-techniques/train.csv')

# Prueba de hipótesis: regresión sobre los precios de la vivienda
[El notebook de kaggle más votado](https://www.kaggle.com/pmarcelino/comprehensive-data-exploration-with-python): interés especial en la sección 5 donde se analizan los supuestos más importantes de regresión lineal ... esto podría ser un marco para la prueba de hipótesis

# Métodos árboles
para llenar datos faltantes.